In [467]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

Importing the data using read_table


In [376]:
ratings = pd.read_table("..\\data\\raw\\\ml-100k\\u.data",names=["user id", "item id","rating", "timestamp"])
movies = pd.read_table("..\\data\\raw\\\ml-100k\\u.item",encoding="latin-1",delimiter="|",names = ["movie id" ,"movie title" ,"release date" ,"video release date","IMDb URL" ,"unknown" ,"Action" ,"Adventure" ,"Animation","Children's" ,"Comedy" ,"Crime" ,"Documentary" ,"Drama","Fantasy","Film-Noir" ,"Horror" ,"Musical" ,"Mystery" ,"Romance" ,"Sci-Fi","Thriller" ,"War" ,"Western"])
genres = pd.read_table("..\\data\\raw\\\ml-100k\\u.genre",delimiter="|", names=["Genre"])
users = pd.read_table("..\\data\\raw\\\ml-100k\\u.user",delimiter="|",names=["user id", "age", "gender", "occupation", "zip code"])


Looking at some summary statistics

In [410]:
ratings.describe()

,user id,item id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [411]:
users.describe()

,user id,age
count,943.000000,943.000000
mean,472.000000,34.051962
std,272.364951,12.192740
min,1.000000,7.000000
25%,236.500000,25.000000
50%,472.000000,31.000000
75%,707.500000,43.000000
max,943.000000,73.000000


Merging all data sets. Here, a row is a rating with the associated movie, movie statistcs, user and user information. We also drop unimportant information here.

In [384]:
 
df = ratings.merge(movies, left_on="item id", right_on="movie id")
df = df.merge(users, left_on="user id",right_on="user id")
df = df.drop(['item id', "movie title", "release date", "video release date", "unknown", "IMDb URL","occupation","gender",  'zip code',"timestamp","age","movie id","rating"], axis = 1)

For the Kmeans regression I'm going to use the number of movies watched in each category to group these users. This will give us users who have the same interests. We can use this information to recommend movies to users that other people in the same cluster as them have seen.

In [385]:
movie_ratings = df.groupby("user id").sum()
movie_ratings

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user id,,,,,,,,,,,,,,,,,,
1,75,42,12,25,91,25,5,107,2,1,13,13,5,44,43,52,25,6
2,10,3,1,4,16,9,0,35,1,2,2,1,4,16,4,12,3,0
3,14,4,0,0,12,10,1,22,0,2,5,2,11,5,8,21,5,0
4,8,4,0,0,4,4,1,6,0,0,1,1,5,3,6,11,2,0
5,56,33,14,29,82,9,0,27,2,1,28,12,3,19,33,19,14,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,18,10,1,1,15,3,0,18,1,0,1,1,1,10,8,12,3,0
940,23,12,2,5,42,7,0,48,0,3,4,7,3,24,19,20,15,0
941,10,7,3,2,7,1,0,5,0,0,0,1,1,1,8,7,1,0


In [386]:
kmeans = KMeans(n_clusters=10,random_state=0).fit_predict(movie_ratings)

In [389]:
kmeans[:100]

array([1, 0, 0, 2, 9, 6, 5, 0, 2, 6, 6, 0, 3, 8, 8, 8, 2, 4, 2, 0, 7, 7,
       9, 0, 0, 8, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 9, 2, 2, 0, 6, 4, 7,
       2, 2, 2, 0, 6, 2, 2, 0, 2, 0, 2, 7, 9, 6, 5, 6, 2, 4, 8, 6, 8, 2,
       2, 2, 0, 9, 2, 8, 0, 2, 0, 0, 0, 2, 0, 2, 0, 9, 9, 0, 4, 2, 1, 2,
       8, 4, 8, 5, 2, 5, 1, 0, 0, 2, 8, 0])

Here we look at how many people from each group watched a movie. In our recommendation algorithm this is what we would use to recommend movies. You are grouped in some cluster using Kmeans, and you want to watch a movie. The algorithm would look at this table and recommend the most watched movie that you haven't seen among the people in your cluster.

In [450]:
users_=ratings["user id"].tolist()
group = []
for user in users_:
    group.append(user_rec.loc[user].group)
ratings.group = group
tmps = pd.get_dummies(ratings.group)
for  tmp in tmps.columns:
    ratings[tmp] = tmps[tmp]
popular = ratings.groupby("item id").sum()

In [451]:
popular 

,user id,rating,timestamp,0,1,2,3,4,5,6,7,8,9
item id,,,,,,,,,,,,,
1,215609,1753,399028021059,94,38,80,6,33,24,43,36,44,54
2,64453,420,115727673079,6,32,0,6,8,20,13,28,7,11
3,41322,273,79400420109,10,15,3,2,10,16,9,8,7,10
4,98125,742,184487964275,19,31,2,7,33,27,27,28,19,16
5,37786,284,75902581757,13,17,5,4,12,14,5,10,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,863,1,889289570,0,0,0,0,0,0,0,0,1,0
1679,863,3,889289491,0,0,0,0,0,0,0,0,1,0
1680,863,2,889289570,0,0,0,0,0,0,0,0,1,0


In [458]:
groupings = MeanShift(bandwidth=2).fit_predict(movie_ratings)

In [459]:
groupings

array([ 75, 631, 536, 700, 136, 344,  33, 220, 854, 350, 266, 528,   1,
       432, 384, 301, 848, 345, 815, 419, 221, 107, 216, 686, 407, 249,
       628, 335, 640, 434, 819, 498, 612, 933, 600, 893, 159, 272, 873,
       713, 568, 197, 154, 190, 614, 703, 930, 538, 237, 942, 647, 648,
       504, 310, 461,  97, 284, 347,  83, 202, 773, 116, 506, 121, 584,
       479, 565, 656, 416, 167, 758, 181, 625, 744, 440, 592, 322, 898,
       771, 869, 621, 293, 179, 465, 178, 835,  80, 872, 580, 288, 262,
        20, 847,  22,  62, 513, 472, 883, 219, 550, 406,  25, 460, 358,
       796, 560, 870, 587,  43, 160, 894, 643, 607, 660, 353, 267, 277,
       539, 122, 541, 424, 816, 812, 571, 263, 551, 519, 273, 783,   7,
       924, 863, 705, 684, 387, 785, 317, 765, 923, 913, 290, 661, 833,
       174,  56, 817, 895, 552, 753, 733, 186, 510, 573, 692, 794, 793,
       449,  57, 303, 364, 543, 439, 837, 330, 622, 752, 559, 383, 585,
       746, 896, 917, 681, 451, 549, 526, 268,  44, 676, 482, 12

Going through the same thing here except we are using Meanshift to determine the groupings as opposed to kmeans

In [471]:
mean_shift = MeanShift().fit_predict(movie_ratings)

In [472]:
mean_shift[:100]

array([ 1,  0,  0,  0, 14, 14,  1,  0,  0, 15, 14,  0,  5,  0,  0,  0,  0,
        1,  0,  0, 15, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, 14, 14, 15,  0,  0,  0,  0, 14,  0,  0,
        0,  0,  0,  0, 14,  0,  0,  1, 15,  0,  1,  0, 14,  0,  0,  0,  0,
        0,  0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 14,  0,  1,
        0, 14,  0,  0, 13,  0,  1,  0,  1, 14,  0,  0,  0, 15,  0],
      dtype=int64)

In [473]:
users_=ratings["user id"].tolist()
group = []
for user in users_:
    group.append(user_rec.loc[user].group)
ratings.group = group
tmps = pd.get_dummies(ratings.group)
for  tmp in tmps.columns:
    ratings[tmp] = tmps[tmp]
popular = ratings.groupby("item id").sum()